In [ ]:
import copy
import time, importlib
import h5py
import numpy as np
import os
import pandas as pd
import scipy, scipy.ndimage
import tqdm
import unyt

In [ ]:
import kalepy as kale

In [ ]:
import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.patheffects as patheffects

In [ ]:
import palettable

In [ ]:
import linefinder.analyze_data.worldlines as a_worldlines
import linefinder.config as l_config

In [ ]:
import galaxy_dive.plot_data.plotting as plotting

In [ ]:
import coolingFunction

In [ ]:
import trove
import verdict
from py2tex import py2tex

In [ ]:
%matplotlib inline

# Parameters

In [ ]:
pm = dict(
    snum = 600,
    ahf_index = 600,
)

In [ ]:
pm = trove.link_params_to_config(
    '/home1/03057/zhafen/papers/Hot-Accretion-in-FIRE/analysis/hot_accretion.trove',
    script_id = 'nb.8',
    variation = 'm12i_md',
    global_variation = 'track_all_recent_stars',
    **pm
)

In [ ]:
sim_name = pm['variation']

In [ ]:
print( sim_name, pm['global_variation'] )

# Load Data

In [ ]:
w = a_worldlines.Worldlines(
    tag = pm['tag'],
    data_dir = pm['data_dirs']['jug.4'],
    halo_data_dir = pm['halo_data_dir'],
    ahf_index = pm['ahf_index'],
)

In [ ]:
w.data_dir

In [ ]:
data_fp = os.path.join( pm['processed_data_dir'], 'summary.hdf5' )
data = verdict.Dict.from_hdf5( data_fp, create_nonexistent=True )

# Analyze

## Calculate Central Indices

In [ ]:
calc_fn = getattr( w, 'calc_{}'.format( pm['central_indices'] ) )
inds = calc_fn(
    lookback_time_max = pm['lookback_time_max'],
    choose_first = pm['choose_first'],
)


In [ ]:
t_tchange_key = pm['central_indices'].split( '_' )[0]
tchange = w.get_data( t_tchange_key )

In [ ]:
fig = plt.figure()
ax = plt.gca()

_ = ax.hist(
    tchange,
    bins = 64,
)

ax.axvline(
    w.get_data( 'time' )[0] - 1.,
    color = pm['background_linecolor'],
    linewidth = 1,
)